In [1]:
#Checking wich GPU is beeing used

!nvidia-smi

Thu Nov  4 13:46:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:05:00.0 Off |                  N/A |
| 29%   26C    P8    18W / 250W |   2287MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:09:00.0 Off |                  N/A |
| 29%   

In [2]:
import pandas as pd
import numpy as np
import datetime
import scipy as sp
import scipy.fftpack
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.io import wavfile
from scipy.fft import fft, ifft, fftfreq
from scipy import signal
import os
import _thread
import pickle


import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda, Compose

import numpy as np
import pandas as pd
import splitfolders as splitfolders

In [3]:
#folder for loading the images
def img_from_lib (filename, data, sr, imagedir):
    #fourier transform on data, save in x
    X = librosa.stft(data)
    #returns absolute value of X as decibel
    Xdb = librosa.amplitude_to_db(abs(X))
    fig = plt.figure()
    #generate the image with axes
    img = librosa.display.specshow(Xdb, sr = sr, x_axis = 'time', y_axis = 'hz', vmin=-60, vmax=20)
    fig.colorbar(img)
    #saves the image a file
    fig.savefig(imagedir + filename.replace('.wav', '') + '_spectro.png')
    plt.close()

In [4]:
def wav_to_img(audiofiledir, imagedir):
    for filename in os.listdir(audiofiledir):
        
        #samplerate and audiodata is retried from audio file
        sr_data, data = wavfile.read(audiofiledir + filename)
        #convert to librosa format
        audiodata, sr = librosa.load(audiofiledir + filename, sr = sr_data)
        #half the samplerate is saved in half_sr
        half_sr = int(sr/2)
        #an amount of zeroes equal to half the sr is saved in arr
        arr = np.zeros(int(half_sr))
        
        audiodata = np.concatenate((audiodata, arr), axis=None)
        img_from_lib(filename, audiodata[:half_sr], sr_data, imagedir)

In [5]:
wav_to_img('./audio/wavfiles/', './images_nosplit/spectrograms/')

In [6]:
#moves & splits data into train, test, val folders. 
splitfolders.ratio("./images_nosplit/", output="./images_splitfolders/", seed=1337, ratio=(.8, .1, .1), group_prefix=None)

Copying files: 10 files [00:00, 3875.36 files/s]


In [11]:
# where to load train,test,val data from
data_dir = 'images_splitfolders/'
#preparing the transform function to transform data into a Tensor object
transform = transforms.Compose([
transforms.ToTensor()]) 
#contents from the 'train' folder is put in train_set
train_set = datasets.ImageFolder(data_dir + 'train', transform=transform) 
test_set = datasets.ImageFolder(data_dir + 'test', transform=transform)
val_set = datasets.Imagefolder(data_dir + 'val', transform=transform)
#train_set is made into a dataloader, shuffled
train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=32, shuffle=True)

#not sure why it throws the "FileNotFoundError" about a ipynb_checkpoints-file. i think its a hidden file that got stuck there.
#but the error doesnt seem to interfere with the rest of the program

FileNotFoundError: Found no valid file for the classes .ipynb_checkpoints. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [12]:
print(train_dataloader)

NameError: name 'train_dataloader' is not defined

In [ ]:
def saveLoader(loader, filename):
    pickle.dump(loader, open(filename, 'wb'))

In [ ]:
saveLoader(rfc, 'rfc_dump')

In [ ]:
#Shutting down the notebook

%%javascript
Jupyter.notebook.session.delete();